<a href="https://colab.research.google.com/github/Ruin9999/sentiment-analysis/blob/main/original_notebooks/sentiment_analysis_part_3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install nltk datasets gensim numpy optuna

In [ ]:
import numpy as np
import re
import nltk
from datasets import load_dataset
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid

import optuna
from typing import List
import torch.optim as optim
from optuna.trial import Trial
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score

import matplotlib.pyplot as plt

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

# Part 0. Dataset Preparation

In [ ]:
# loading the dataset from the library
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
# check the sizes of each dataset
train_size = len(train_dataset)
validation_size = len(validation_dataset)
test_size = len(test_dataset)

print(f"Training dataset size: {train_size}")
print(f"Validation dataset size: {validation_size}")
print(f"Test dataset size: {test_size}")

Training dataset size: 8530
Validation dataset size: 1066
Test dataset size: 1066


In [ ]:
# view an example from each dataset
print("Train Dataset")
print(train_dataset.features)
print(train_dataset[0])

print("Test Dataset")
print(test_dataset.features)
print(test_dataset[0])

print("Validation Dataset")
print(validation_dataset.features)
print(validation_dataset[0])

Train Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
Test Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .', 'label': 1}
Validation Dataset
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
{'text': 'compassionately explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children .', 'label': 1}


# Part 1. Preparing Word Embeddings

### Preprocessing

In [ ]:
def preprocessing(text):

    # remove any other special characters but keep the general ones for potential sentiment usage
    text = re.sub(r'[^a-zA-Z0-9\'\!\?\.]', ' ', text)

    # replace multiple spaces with one space only
    text = re.sub(r'\s+', ' ', text)

    # remove leading and trailing whitespace to avoid unnecessary inconsistency
    text = text.strip()

    # convert the text to lowercase
    text = text.lower()


    return text

# apply the preprocessing function to the 'text' column of each dataset
train_dataset = train_dataset.map(lambda x: {'text': preprocessing(x['text'])})
validation_dataset = validation_dataset.map(lambda x: {'text': preprocessing(x['text'])})
test_dataset = test_dataset.map(lambda x: {'text': preprocessing(x['text'])})

# an example of the processed text
print("Train Dataset Example:")
print(train_dataset[0])

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Train Dataset Example:
{'text': "the rock is destined to be the 21st century's new conan and that he's going to make a splash even greater than arnold schwarzenegger jean claud van damme or steven segal .", 'label': 1}


In [ ]:
# tokenization
# empty list to store the resulting sentences
tokenized_sentences = []

for text in train_dataset['text']:
    # Tokenize the text and append the tokenized sentence to the list
    tokenized_sentences.append(word_tokenize(text))

### (a) Size of vocabulary in training data

In [ ]:
# empty set for storing unique words
original_vocab = set()

for sentence in tokenized_sentences:
    for word in sentence:
        # add each word in the sentence to the words set
        original_vocab.add(word)

print(f"(a) The size of vocabulary formed in the training data is {len(original_vocab)}")

(a) The size of vocabulary formed in the training data is 16683


### (b) Number of OOV in the training data

In [ ]:
# adjust the parameters for word2vec
vector_size = 100 # Dimensionality of the word vectors
window = 3 # Maximum distance between the current and predicted word within a sentence
min_count = 2 # Ignores all words with total frequency lower than this
workers = 4 # CPU cores
sg = 1 # 1 for skip-gram, 0 for CBOW
epochs = 5

# train the word2vec model
word2vec_model = Word2Vec(
    sentences = tokenized_sentences,
    vector_size = vector_size,
    window = window,
    min_count = min_count,
    workers = workers,
    epochs = epochs)

# variable to store model's vocab list
word2vec_vocab = set(word2vec_model.wv.key_to_index)

# Calculate OOV words by comparing the original vocab and Word2Vec vocab
oov_words = original_vocab - word2vec_vocab

print(f"(b) Number of OOV words in the training data is {len(oov_words)} when the minimum threshold for each word is {min_count}")

(b) Number of OOV words in the training data is 7866 when the minimum threshold for each word is 2


### (c) Mitigating OOV

In [ ]:
# define the UNK and PAD token
UNK_TOKEN = '<UNK>'
PAD_TOKEN = '<PAD>'

def replace_oov_words(tokenized_sentences,vocab):
    # Replace all OOV words with <UNK>
    # process each sentence in the tokenized_sentences list
    for i, sentence in enumerate(tokenized_sentences):
        # empty list to store the current processed sentence
        processed_sentence = []
        for word in sentence:
            if word in vocab:
                # if the current word is in the model's vocab, keep it as it is
                processed_sentence.append(word)
            else:
                # otherwise, replace the word with UNK
                processed_sentence.append(UNK_TOKEN)

        # update the sentence in the original tokenized_sentences list
        tokenized_sentences[i] = processed_sentence

    return tokenized_sentences

### Embedding matrix

In [ ]:
# empty set for storing unique words
final_vocab = set()

for sentence in tokenized_sentences:
    for word in sentence:
        # add each word in the sentence to the final_vocab set
        final_vocab.add(word)

# add 'UNK' and '<PAD>' to the vocabulary
final_vocab.add(UNK_TOKEN)
final_vocab.add(PAD_TOKEN)

# create the dictionary that maps each word in final_vocab to a unique index
word_to_index = {word: i for i, word in enumerate(final_vocab)}

embedding_dim = word2vec_model.vector_size

# initialize embedding matrix with number of vocab and embedding dimension
embedding_matrix = np.zeros((len(word_to_index), embedding_dim))

# fill the embedding matrix with the corresponding word vectors
for word, i in word_to_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]
    elif word == PAD_TOKEN:
        # give padding token a zero vector to have no impact on the word semantics
        embedding_matrix[i] = np.zeros(embedding_dim)
    else:
        # use average vector for unknown words
        embedding_matrix[i] = np.mean(word2vec_model.wv.vectors, axis=0)

print(f"Shape of embedding matrix: {embedding_matrix.shape}")

Shape of embedding matrix: (16685, 100)


In [ ]:
# convert word to indices
def words_to_indices(sentence, word_to_index):
    return [word_to_index.get(word, word_to_index[UNK_TOKEN]) for word in sentence.split()]

train_X = [words_to_indices(sentence, word_to_index) for sentence in train_dataset['text']]
train_y = train_dataset['label']
val_X = [words_to_indices(sentence, word_to_index) for sentence in validation_dataset['text']]
val_y = validation_dataset['label']
test_X = [words_to_indices(sentence, word_to_index) for sentence in test_dataset['text']]
test_y = test_dataset['label']

def create_dataloader(X, y, batch_size=16, shuffle=True):
    X_tensor = [torch.tensor(seq, dtype=torch.long) for seq in X]
    X_padded = pad_sequence(X_tensor, batch_first=True, padding_value=word_to_index[PAD_TOKEN])
    y_tensor = torch.tensor(y, dtype=torch.long)
    dataset = TensorDataset(X_padded, y_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

train_dataloader = create_dataloader(train_X, train_y, shuffle=True)
val_dataloader = create_dataloader(val_X, val_y, shuffle=False)
test_dataloader = create_dataloader(test_X, test_y, shuffle=False)

# convert embedding_matrix to tensor
embedding_matrix = torch.FloatTensor(embedding_matrix)

# **Part 3.4 Model Training & Evaluation - CNN**

In [ ]:
torch.manual_seed(42)

## Creating a CNN architecture

In [ ]:
class SentimentCNN(nn.Module):
  def __init__(self,
               embedding_dim: int,
               embedding_matrix: torch.FloatTensor,
               pad_idx: int,
               num_classes: int,
               num_filters: int,
               filter_sizes: List[int],
               dropout: float = 0.5,
               freeze_embeddings: bool = True,
               hidden_dim: int = 128):
    super(SentimentCNN, self).__init__()

    # Load pretrained embeddings
    embedding_tensor = torch.FloatTensor(embedding_matrix)
    self.embedding = nn.Embedding.from_pretrained(embedding_tensor, padding_idx=pad_idx, freeze=freeze_embeddings)

    self.convs = nn.ModuleList([
        nn.Conv2d(1, num_filters, (fs, embedding_dim)) for fs in filter_sizes
    ])

    self.dropout = nn.Dropout(dropout)
    self.fc1 = nn.Linear(len(filter_sizes) * num_filters, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, num_classes)

  def forward(self, x):
    x = self.embedding(x)
    x = x.unsqueeze(1)

    conv_outputs = []
    for conv in self.convs:
        conv_out = torch.relu(conv(x)).squeeze(3)
        pool_out = torch.max_pool1d(conv_out, conv_out.shape[2]).squeeze(2)
        conv_outputs.append(pool_out)


    x = torch.cat(conv_outputs, dim=1)
    x = self.dropout(x)
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.dropout(x)
    x = self.fc2(x)

    return x

In [ ]:
def train_epoch(model, train_dataloader, optimizer, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for batch in train_dataloader:
      texts, labels = batch
      texts, labels = texts.to(device), labels.to(device)

      optimizer.zero_grad()
      predictions = model(texts)

      loss = criterion(predictions, labels)
      acc = (predictions.argmax(1) == labels).float().mean()

      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()
      epoch_acc += acc.item()

    val_loss, val_acc = evaluate(model, val_dataloader, criterion)

    return (epoch_loss / len(train_dataloader), epoch_acc / len(train_dataloader),
            val_loss, val_acc)

In [ ]:
def evaluate(model, val_dataloader, criterion):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model.eval()
  epoch_loss = 0
  epoch_acc = 0

  with torch.no_grad():
    for batch in val_dataloader:
      texts, labels = batch
      texts, labels = texts.to(device), labels.to(device)

      predictions = model(texts)

      loss = criterion(predictions, labels)
      acc = (predictions.argmax(1) == labels).float().mean()

      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss / len(val_dataloader), epoch_acc / len(val_dataloader)

In [ ]:
def test_model(model, test_loader):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  model.eval()
  total_loss = 0
  total_acc = 0

  with torch.no_grad():
      for texts, labels in test_loader:
        texts, labels = texts.to(device), labels.to(device)

        outputs = model(texts)

        loss = criterion(outputs, labels)
        acc = (outputs.argmax(1) == labels).float().mean()

        total_loss += loss.item()
        total_acc += acc.item()

  return total_loss / len(test_loader), total_acc / len(test_loader)

In [ ]:
def objective(trial: Trial, train_dataloader, val_dataloader, freeze_embeddings):
    # Define hyperparameters to optimize
    params = {
        'num_filters': trial.suggest_int('num_filters', 64, 256),
        'filter_sizes': [2, 3, 4],  # Could also be optimized if desired
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'epochs': trial.suggest_int('epochs', 1, 10),
    }

    # Initialize model, criterion, and optimizer
    model = SentimentCNN(
        embedding_dim=embedding_matrix.shape[1],
        pad_idx=word_to_index[PAD_TOKEN],
        embedding_matrix=embedding_matrix,
        num_classes=2,
        num_filters=params['num_filters'],
        filter_sizes=params['filter_sizes'],
        dropout=params['dropout'],
        hidden_dim=128,
        freeze_embeddings=freeze_embeddings,
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training loop
    n_epochs = params['epochs']
    best_val_acc = 0

    for epoch in range(n_epochs):
        train_loss, train_acc, val_loss, val_acc = train_epoch(model, train_dataloader, optimizer, criterion)

        # Report intermediate value
        trial.report(val_acc, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.TrialPruned()

        if val_acc > best_val_acc:
            best_val_acc = val_acc

    return best_val_acc

## Hyperparameter searching

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, train_dataloader, val_dataloader, freeze_embeddings=True), n_trials=100)

frozen_best_params = study.best_params
print(f"Best parameters for frozen model: {frozen_best_params}")
print(f"Best validation accuracy frozen model: {study.best_value}")

[I 2024-11-08 17:49:26,207] A new study created in memory with name: no-name-9693f436-5e96-46e2-9fbf-bb921c638e90
[I 2024-11-08 17:49:31,101] Trial 0 finished with value: 0.5574626869230128 and parameters: {'num_filters': 153, 'dropout': 0.30113956397556807, 'learning_rate': 0.0004998319826209201, 'epochs': 3}. Best is trial 0 with value: 0.5574626869230128.
[I 2024-11-08 17:49:43,007] Trial 1 finished with value: 0.5315298509241929 and parameters: {'num_filters': 162, 'dropout': 0.1234218957601657, 'learning_rate': 0.002996804497721886, 'epochs': 7}. Best is trial 0 with value: 0.5574626869230128.
[I 2024-11-08 17:49:48,908] Trial 2 finished with value: 0.5546641794603262 and parameters: {'num_filters': 235, 'dropout': 0.20416636253499829, 'learning_rate': 0.0016989888052023254, 'epochs': 3}. Best is trial 0 with value: 0.5574626869230128.
[I 2024-11-08 17:49:55,287] Trial 3 finished with value: 0.5639925376692815 and parameters: {'num_filters': 247, 'dropout': 0.1280128136330709, 'le

Best parameters for frozen model: {'num_filters': 198, 'dropout': 0.2987211042790332, 'learning_rate': 0.0009928900184418767, 'epochs': 6}
Best validation accuracy frozen model: 0.5714552242364457


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, train_dataloader, val_dataloader, freeze_embeddings=False), n_trials=100)

unfrozen_best_params = study.best_params
print(f"Best parameters for unfrozen model: {unfrozen_best_params}")
print(f"Best validation accuracy unfrozen model: {study.best_value}")

[I 2024-11-08 17:55:23,081] A new study created in memory with name: no-name-1d809b80-1d5c-42f1-9f65-61926aee02a1
[I 2024-11-08 17:55:28,864] Trial 0 finished with value: 0.7513059706830266 and parameters: {'num_filters': 149, 'dropout': 0.14345616978086953, 'learning_rate': 0.0003286062830008199, 'epochs': 3}. Best is trial 0 with value: 0.7513059706830266.
[I 2024-11-08 17:55:31,590] Trial 1 finished with value: 0.5404850747158279 and parameters: {'num_filters': 65, 'dropout': 0.4238324103737314, 'learning_rate': 0.00038097983777767654, 'epochs': 1}. Best is trial 0 with value: 0.7513059706830266.
[I 2024-11-08 17:55:47,675] Trial 2 finished with value: 0.7460820897301631 and parameters: {'num_filters': 86, 'dropout': 0.19227788214472633, 'learning_rate': 0.0006052821451036357, 'epochs': 8}. Best is trial 0 with value: 0.7513059706830266.
[I 2024-11-08 17:56:03,920] Trial 3 finished with value: 0.7281716421468934 and parameters: {'num_filters': 188, 'dropout': 0.15777095078417888, 'l

Best parameters for unfrozen model: {'num_filters': 232, 'dropout': 0.1997304944885224, 'learning_rate': 0.000962377658709009, 'epochs': 4}
Best validation accuracy unfrozen model: 0.7699626871009371


## Testing the *optimized* models

In [ ]:
frozen_embeddings_model = SentimentCNN(
    embedding_dim=embedding_matrix.shape[1],
    pad_idx=word_to_index[PAD_TOKEN],
    embedding_matrix=embedding_matrix,
    num_classes=2,
    num_filters=frozen_best_params['num_filters'],
    filter_sizes=[2, 3, 4],
    dropout=frozen_best_params['dropout'],
    freeze_embeddings=True,
    hidden_dim=128
).to("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(frozen_embeddings_model.parameters(), lr=frozen_best_params['learning_rate'])

# Train final model
n_epochs = frozen_best_params['epochs']
for epoch in range(n_epochs):
    train_loss, train_acc, val_loss, val_acc = train_epoch(frozen_embeddings_model, train_dataloader, optimizer, criterion)
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f}, Val Loss: {val_loss}, Val Acc: {val_acc}')

# Evaluate final model
test_loss, test_acc = test_model(frozen_embeddings_model, test_dataloader)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.3f}')

Epoch: 01, Train Loss: 0.696, Train Acc: 0.502, Val Loss: 0.6922459264299763, Val Acc: 0.5013059706830266
Epoch: 02, Train Loss: 0.694, Train Acc: 0.516, Val Loss: 0.6920084633044342, Val Acc: 0.5686567167737591
Epoch: 03, Train Loss: 0.691, Train Acc: 0.527, Val Loss: 0.6915253783339885, Val Acc: 0.5082089554018049
Epoch: 04, Train Loss: 0.690, Train Acc: 0.533, Val Loss: 0.6909492718639658, Val Acc: 0.549813432924783
Epoch: 05, Train Loss: 0.688, Train Acc: 0.544, Val Loss: 0.6885136846286147, Val Acc: 0.5518656719976397
Epoch: 06, Train Loss: 0.688, Train Acc: 0.543, Val Loss: 0.6909653419878945, Val Acc: 0.5222014927152377
Test Loss: 0.690, Test Acc: 0.536


In [ ]:
unfrozen_embeddings_model = SentimentCNN(
    embedding_dim=embedding_matrix.shape[1],
    pad_idx=word_to_index[PAD_TOKEN],
    embedding_matrix=embedding_matrix,
    num_classes=2,
    num_filters=unfrozen_best_params['num_filters'],
    filter_sizes=[2, 3, 4],
    dropout=unfrozen_best_params['dropout'],
    freeze_embeddings=False,
    hidden_dim=128
).to("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(unfrozen_embeddings_model.parameters(), lr=unfrozen_best_params['learning_rate'])

# Train final model
n_epochs = unfrozen_best_params['epochs']
for epoch in range(n_epochs):
    train_loss, train_acc, val_loss, val_acc = train_epoch(unfrozen_embeddings_model, train_dataloader, optimizer, criterion)
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f}, Val Loss: {val_loss}, Val Acc: {val_acc}')

# Evaluate final model
test_loss, test_acc = test_model(unfrozen_embeddings_model, test_dataloader)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.3f}')

Epoch: 01, Train Loss: 0.691, Train Acc: 0.529, Val Loss: 0.6729423030098872, Val Acc: 0.5805970150143352
Epoch: 02, Train Loss: 0.480, Train Acc: 0.767, Val Loss: 0.5736786184915855, Val Acc: 0.7149253736681013
Epoch: 03, Train Loss: 0.187, Train Acc: 0.927, Val Loss: 0.6550819124303647, Val Acc: 0.7492537311653593
Epoch: 04, Train Loss: 0.068, Train Acc: 0.976, Val Loss: 1.081089185022596, Val Acc: 0.7425373134328358
Test Loss: 1.000, Test Acc: 0.730


In [ ]:
# Test the model
frozen_test_loss, frozen_test_acc = test_model(frozen_embeddings_model, test_dataloader)
unfrozen_test_loss, unfrozen_test_acc = test_model(unfrozen_embeddings_model, test_dataloader)

print(f'Frozen loss: {frozen_test_loss}, Frozen acc: {frozen_test_acc}')
print(f'Unfrozen loss: {unfrozen_test_loss}, Unfrozen acc: {unfrozen_test_acc}')

Frozen loss: 0.6896988242419798, Frozen acc: 0.535820896056161
Unfrozen loss: 1.0004538928172482, Unfrozen acc: 0.7296641789265533


# **Using unfrozen embeddings + updated OOV handling from part 3.2**

In [ ]:
tokenized_sentences = replace_oov_words(tokenized_sentences, word2vec_vocab)

# empty set for storing unique words
final_vocab = set()

for sentence in tokenized_sentences:
    for word in sentence:
        # add each word in the sentence to the final_vocab set
        final_vocab.add(word)

# add 'UNK' and '<PAD>' to the vocabulary
final_vocab.add(UNK_TOKEN)
final_vocab.add(PAD_TOKEN)

# create the dictionary that maps each word in final_vocab to a unique index
word_to_index = {word: i for i, word in enumerate(final_vocab)}

embedding_dim = word2vec_model.vector_size

# initialize embedding matrix with number of vocab and embedding dimension
embedding_matrix = np.zeros((len(word_to_index), embedding_dim))

# fill the embedding matrix with the corresponding word vectors
for word, i in word_to_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]
    else:
        # (option 1) random initialization for unknown words
        embedding_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim,))
        # (option 2) use average vector for unknown words
        # embedding_matrix[i] = np.mean(word2vec_model.wv.vectors, axis=0)

print(f"Shape of embedding matrix: {embedding_matrix.shape}")

Shape of embedding matrix: (8819, 100)


In [ ]:

train_X = [
    words_to_indices(sentence, word_to_index) for sentence in train_dataset["text"]
]
train_y = train_dataset["label"]
val_X = [
    words_to_indices(sentence, word_to_index) for sentence in validation_dataset["text"]
]
val_y = validation_dataset["label"]
test_X = [
    words_to_indices(sentence, word_to_index) for sentence in test_dataset["text"]
]
test_y = test_dataset["label"]



train_dataloader = create_dataloader(train_X, train_y, shuffle=True)
val_dataloader = create_dataloader(val_X, val_y, shuffle=False)
test_dataloader = create_dataloader(test_X, test_y, shuffle=False)

# convert embedding_matrix to tensor
embedding_matrix = torch.FloatTensor(embedding_matrix)

In [ ]:
final_unfrozen_embeddings_model = SentimentCNN(
    embedding_dim=embedding_matrix.shape[1],
    pad_idx=word_to_index[PAD_TOKEN],
    embedding_matrix=embedding_matrix,
    num_classes=2,
    num_filters=unfrozen_best_params['num_filters'],
    filter_sizes=[2, 3, 4],
    dropout=unfrozen_best_params['dropout'],
    freeze_embeddings=False,
    hidden_dim=128
).to("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(final_unfrozen_embeddings_model.parameters(), lr=unfrozen_best_params['learning_rate'])

# Train final model
n_epochs = unfrozen_best_params['epochs']
for epoch in range(n_epochs):
    train_loss, train_acc, val_loss, val_acc = train_epoch(final_unfrozen_embeddings_model, train_dataloader, optimizer, criterion)
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f}')

# Evaluate final model
test_loss, test_acc = evaluate(final_unfrozen_embeddings_model, test_dataloader, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.3f}')

Epoch: 01, Train Loss: 0.695, Train Acc: 0.511
Epoch: 02, Train Loss: 0.606, Train Acc: 0.660
Epoch: 03, Train Loss: 0.316, Train Acc: 0.866
Epoch: 04, Train Loss: 0.151, Train Acc: 0.944
Test Loss: 0.840, Test Acc: 0.731


# **Trying to improve the model**

In [ ]:
class ImprovedSentimentCNN(nn.Module):
  def __init__(self,
               embedding_dim: int,
               embedding_matrix: torch.FloatTensor,
               pad_idx: int,
               num_classes: int,
               num_filters: int,
               filter_sizes: List[int],
               dropout: float = 0.5,
               freeze_embeddings: bool = True,
               hidden_dim1: int = 128,
               hidden_dim2: int = 64,
               hidden_dim3: int = 32):
    super(ImprovedSentimentCNN, self).__init__()

    # Load pretrained embeddings
    embedding_tensor = torch.FloatTensor(embedding_matrix)
    self.embedding = nn.Embedding.from_pretrained(embedding_tensor, padding_idx=pad_idx, freeze=freeze_embeddings)

    self.convs = nn.ModuleList([
        nn.Conv2d(1, num_filters, (fs, embedding_dim)) for fs in filter_sizes
    ])

    # Batch Normalization
    self.bn = nn.BatchNorm1d(num_filters)

    # Self Attention
    self.attention = nn.MultiheadAttention(embed_dim=num_filters * len(filter_sizes), num_heads=2, dropout=dropout)

    self.dropout = nn.Dropout(dropout)
    self.fc1 = nn.Linear(len(filter_sizes) * num_filters, hidden_dim1)
    self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
    self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
    self.fc4 = nn.Linear(hidden_dim3, num_classes)

  def forward(self, x):
    x = self.embedding(x)
    x = x.unsqueeze(1)

    conv_outputs = []
    for conv in self.convs:
        conv_out = torch.relu(conv(x)).squeeze(3)
        # conv_out = self.bn(conv_out) # (Reduced Performance)
        pool_out = torch.max_pool1d(conv_out, conv_out.shape[2]).squeeze(2)
        conv_outputs.append(pool_out)

    x = torch.cat(conv_outputs, dim=1)

    # Applying attention (Reduced performance)
    # x = x.unsqueeze(1)
    # x, _ = self.attention(x, x, x)
    # x = x.squeeze(1)

    # Fully connected layers
    x = self.dropout(x)
    x = self.fc1(x)
    x = torch.relu(x)
    x = self.dropout(x)
    x = self.fc2(x)
    x = torch.relu(x)
    x = self.dropout(x)
    x = self.fc3(x)
    x = torch.relu(x)
    x = self.dropout(x)
    x = self.fc4(x)

    return x

In [ ]:
attention_model = ImprovedSentimentCNN(
    embedding_dim=embedding_matrix.shape[1],
    pad_idx=word_to_index[PAD_TOKEN],
    embedding_matrix=embedding_matrix,
    num_classes=2,
    num_filters=200,
    filter_sizes=[2, 3, 4],
    dropout=0.17269878819772821,
    freeze_embeddings=False,
).to("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(attention_model.parameters(), lr=0.003259879759370297)

# Train final model
n_epochs = 5 # We decrease the number of epochs because we have so many fully connected layers. Also observed to overfit when around 7.
for epoch in range(n_epochs):
    train_loss, train_acc, val_loss, val_acc = train_epoch(attention_model, train_dataloader, optimizer, criterion)
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f}, Val Loss: {val_loss}, Val Acc: {val_acc}')

# Evaluate final model
test_loss, test_acc = test_model(attention_model, test_dataloader)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.3f}')

In [ ]:
sentences = list([
    "A masterpiece! The storyline was captivating, and the acting was top-notch. Definitely worth a watch.",
    "Loved every minute! The visuals and soundtrack were absolutely stunning.",
    "A fresh take on a classic genre. Brilliantly executed and highly entertaining.",
    "Exceeded my expectations! The plot twists kept me on the edge of my seat.",
    "Incredible character development and a powerful message. One of the best films of the year.",
    "A true cinematic experience. Beautifully shot and wonderfully acted.",
    "Great balance of humor and drama, with a cast that brings it to life perfectly.",
    "So much fun to watch! The pacing was just right, and I was thoroughly entertained.",
    "An inspiring story with unforgettable performances. Left me feeling emotional and fulfilled.",
    "It’s rare to find a movie that hits every mark, but this one does it effortlessly.",
    "A complete waste of time. The plot was all over the place, and the characters were flat.",
    "So disappointing. I had high hopes, but it was just poorly executed from start to finish.",
    "The acting was cringe-worthy, and the dialogue was painfully unrealistic.",
    "The storyline was predictable, and it felt like every scene was dragging on.",
    "I struggled to stay awake. The pacing was terrible, and there was zero suspense.",
    "Couldn’t finish it. The movie was boring, with no real plot development.",
    "This was one of the worst movies I’ve ever seen. Nothing made sense.",
    "An absolute disaster. Poor writing, poor acting, and poor everything else.",
    "Felt like a low-budget student film. The special effects were laughable.",
    "Completely overrated. I don’t get the hype; it was dull and uninspired.",
])
sentences = [words_to_indices(sentence, word_to_index) for sentence in sentences]
labels = [1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]

manual_test_dataset = create_dataloader(sentences, labels, shuffle=False)

test_loss, test_acc = test_model(attention_model, manual_test_dataset)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.3f}')

# **Loading and Saving models**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch

torch.save(frozen_embeddings_model.state_dict(), '/content/drive/MyDrive/frozen_embeddings_model.pth')
torch.save(unfrozen_embeddings_model.state_dict(), '/content/drive/MyDrive/unfrozen_embeddings_model.pth')
torch.save(attention_model.state_dict(), '/content/drive/MyDrive/attention_model.pth')

In [ ]:
frozen_embeddings_model = torch.load("/content/drive/MyDrive/frozen_embeddings_model.pth")
unfrozen_embeddings_model = torch.load("/content/drive/MyDrive/unfrozen_embeddings_model.pth")
attention_model = torch.load("/content/drive/MyDrive/attention_model.pth")